# Иерархическая кластеризация точек на прямой

1. Проверяем, что точки действительно лежат на одной прямой.
2. Сортируем точки и считаем расстояния до соседей. Отсортировав полученные расстояния, получим порядок объединения в кластеры, который не измениться т.к. для объединения используется метод одиночной связи (single linkage).
3. Продолжаем процесс объединения до тех пор пока не достигнем либо заданного числа кластеров, либо порога.

In [1]:
import sys
import numpy as np
from operator import itemgetter

In [2]:
class HierarchicalClustering:
    def __init__(self, n_clusters=None, threshold=None):
        # Проверяем, что задано что-то одно либо n_clusters, либо threshold
        assert (n_clusters is None or threshold is None)
        assert not(n_clusters is None and threshold is None), 'n_clusters or threshold must be set.'
        if n_clusters is not None:
            self.n_clusters = n_clusters
            if self.n_clusters <= 0:
                raise Exception('Wrong number of clusters. Number of clusters must be greater than zero.')
            self.threshold = sys.maxsize
        if threshold is not None:
            self.threshold = threshold
            self.n_clusters = -sys.maxsize
    
    def init_points(self, data):
        self.data = data
        self.check_on_line()
        self.sort()
        
    def dist(self, point_1, point_2):
        x_1, y_1 = point_1
        x_2, y_2 = point_2
        return  ((x_1 - x_2) ** 2 + (y_1 - y_2) ** 2) ** 0.5
    
    def check_on_line(self):
        # Проведем прямую через первые две точки
        x_1, y_1 = self.data[0]
        x_2, y_2 = self.data[1]
        def on_this_line(x, y):
            if y_1 == y_2:
                return y == y_1
            if x_1 == x_2:
                return x == x_1
            return (x - x_1) / (x_2 - x_1) == (y - y_1) / (y_2 - y_1)
        # Проверим, чо все остальные точки принадлежат построенной линии 
        for point in self.data[2:]:
            if not on_this_line(*point):
                raise Exception('Рoints don\'t lie on one line.')
            
    def sort(self):
        self.initial_positions, self.data = zip(*sorted(enumerate(self.data), key=itemgetter(1)))
        self.data = list(self.data)

    def fit_transform(self, data):
        self.init_points(data)
        dists, unions = [], []
        for i, val in enumerate(self.data[:-1]):
            dists.append(self.dist(val, self.data[i+1]))
            unions.append([i, i + 1])
        unions, dists = np.array(unions), np.array(dists)
        inds = np.argsort(dists)
        unions, dists = unions[inds], dists[inds]
        
        clusters = {i:{i} for i in range(len(self.data))}
        points_clusters = np.array([i for i in range(len(self.data))])
        
        for pair_id, pair in enumerate(unions):
            if len(clusters) <= self.n_clusters or dists[pair_id] >= self.threshold:
                break
            point_1, point_2 = pair
            cl_1, cl_2 = points_clusters[point_1], points_clusters[point_2]
            points_clusters[points_clusters == cl_2] = cl_1
            clusters[cl_1] = clusters[cl_1].union(clusters[cl_2])
            clusters.pop(cl_2)

        used_clust = 0
        for i, val in enumerate(points_clusters):
            points_clusters[i] = used_clust
            if  i + 1 != len(points_clusters) and val != points_clusters[i + 1]:
                used_clust +=1       
                
        points_clusters = points_clusters[np.argsort(self.initial_positions)]
        return points_clusters.tolist()

In [3]:
points = [(-1, -1), (2, 2), (0, 0), (5.25, 5.25), (3, 3)]
clust = HierarchicalClustering(n_clusters=3)
print("Points:", points)
print("Lables:", clust.fit_transform(points))

Points: [(-1, -1), (2, 2), (0, 0), (5.25, 5.25), (3, 3)]
Lables: [0, 1, 0, 2, 1]


Данный алгоритм будет плохо работать в том случае, когда соседние точки находятся на одинаковом расстоянии друг от друга.
В таком случае постоенная дендрограмма будет похожа на лесенку, а полученные кластеры не будут представлять интереса.